In [2]:
!pip install sentence_transformers
!pip install wikipedia
# !wget -c "https://figshare.com/ndownloader/files/13774556"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.7 MB/s 
     |████████████████████████████████| 4.7 MB 40.4 MB/s 
     |████████████████████████████████| 1.3 MB 8.0 MB/s 
     |████████████████████████████████| 101 kB 6.0 MB/s 
     |████████████████████████████████| 596 kB 48.8 MB/s 
     |████████████████████████████████| 6.6 MB 34.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=2c6ed6ba9964807f716f3cfcefe15c9c4239330030f9c8f36dc9940a05a5555a
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wh

In [3]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from pprint import pprint
from nltk.util import ngrams
import wikipedia as wiki
import gensim
import re
import nltk
from nltk import tokenize
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


model = SentenceTransformer('bert-base-nli-mean-tokens')
nlp = en_core_web_sm.load()
nltk.download('punkt')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Use n-grams for search

In [4]:
def keywords(sentence):

    doc = nlp(sentence)

    results = ([(X.text, X.label_)[0] for X in doc.ents])
    n_gram=[]

    for i in range(2, 5):
        n_gram.extend(ngrams(sentence.split(), i))

    results.extend([' '.join(i) for i in n_gram])
    return list(set(results))

In [5]:
def clean_wiki_text(text):
    text = re.sub(r'==.+==', '.', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\t', ' ', text)
    text = re.sub(r'\[[0-9]+\]', ' ', text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(r'\. \.', '.', text)
    return text

In [6]:
def content(claim, results):
    sentences = []
    found=[]
    for i in results:
        try :
            current_page = wiki.page(i)
            if current_page not in found:
                found.append(current_page)
        except :
            continue
    titles=[i.title for i in found]
    titles=[i[0] for i in topNSimilar(claim, titles)]
    for i in found:
        if i.title not in titles:
            found.remove(i)

    for i in found:
        current_content = i.content
        sentences.extend(tokenize.sent_tokenize(clean_wiki_text(current_content)))
    return sentences

In [7]:
def topNSimilar(claim, sentList, n=5):
    distList=[]
    for i in sentList:
        document_term_matrix = TfidfVectorizer().fit_transform([i, claim])
        dist = document_term_matrix * document_term_matrix.transpose()
        distance = dist.toarray()[0][1]

        if len(distList)<=n:
            distList.append([i, distance])
        else:
            distList=sorted(distList, key=lambda x: x[1], reverse=True)
            if distance>distList[-1][1]:
                distList.pop()
                distList.append([i, distance])
    return sorted(distList, key=lambda x: x[1], reverse=True)

In [ ]:
def topNBert(claim, res):
    topNFacts=[i[0] for i in res]
    topNScore=[cosine_similarity( [model.encode(claim)], [model.encode(i)] )[0][0] for i in topNFacts]
    topN=zip(topNFacts, topNScore)
    return [list(i) for i in list(topN)]

In [8]:
claim = 'Donald Trump is the president of the united states'

In [9]:
topNBert(claim, topNSimilar(claim, content(claim, keywords(claim)), 5))

/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [ ]:
def truthfulness(claim, response):
    

In [10]:
response

[['The president of the United States (POTUS) is the head of state and head of government of the United States of America.',
  0.6366237536513468],
 ['The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.',
  0.6009890730477123],
 ['The president has the power to nominate federal judges, including members of the United States courts of appeals and the Supreme Court of the United States.',
  0.5820313483572557],
 ['This is the only component in the inauguration ceremony mandated by the Constitution: I do solemnly swear (or affirm) that I will faithfully execute the Office of President of the United States, and will to the best of my ability, preserve, protect, and defend the Constitution of the United States.',
  0.5691980928865016],
 ['The position of the United States as the leading member of NATO, and the country\'s strong relationships with other wealthy or democratic nations like those comprising the Eu

In [ ]:
# import nltk
# 


In [ ]:
res=content(claim, keywords(claim))

In [ ]:
res[1]

['Vande Mataram',
 'Written By',
 'Bankim Chandra Chatterjee',
 'It Was Written',
 'Sarat Chandra Chattopadhyay']

In [27]:
a=''
b=''

In [28]:
from sklearn.metrics.pairwise import cosine_similarity
# for i in response:
sim_arr=cosine_similarity( [model.encode(a)], [model.encode(b)] )[0][0]
print(sim_arr)

0.5572978


In [ ]:
def topNSimilar(claim, sentList, n=5):
    distList=[]
    for i in sentList:
        document_term_matrix = TfidfVectorizer().fit_transform([i, claim])
        dist = document_term_matrix * document_term_matrix.transpose()
        distance = dist.toarray()[0][1]

        if len(distList)<=n:
            distList.append([i, distance])
        else:
            distList=sorted(distList, key=lambda x: x[1], reverse=True)
            if distance>distList[-1][1]:
                distList.pop()
                distList.append([i, distance])
    return sorted(distList, key=lambda x: x[1], reverse=True)

In [43]:
def topNBert(claim, res):
    topNFacts=[i[0] for i in res]
    topNScore=[cosine_similarity( [model.encode(claim)], [model.encode(i)] )[0][0] for i in topNFacts]
    topN=zip(topNFacts, topNScore)
    return [list(i) for i in list(topN)]

In [46]:
claim = 'Ooga booga'

In [47]:
topNBert(claim, topNSimilar(claim, content(claim, keywords(claim)), 5))

[['Ooga Booga at MobyGames', 0.7149534],
 ['Ooga Booga received "favorable" reviews according to the review aggregation website Metacritic.',
  0.46006805],
 ['Ooga Booga was developed by Visual Concepts and published by Sega.',
  0.44086593],
 ['Some of them include Death (a floating skeleton), Abe (former U.S. President Abraham Lincoln), Superguy (superhero), Joseph (a caveman who says "ooga booga") and Dwarf (a dwarf).',
  0.32150775],
 ['Ooga Booga is an online multiplayer video game for the Dreamcast, focusing on the combat of "Kahunas" using thrown shrunken heads, riding animals, staffs, or using spells.',
  0.25430715],
 ['The storyline is that Ooga Booga is a volcano goddess that creates islands, and has leaders of tribes, the Kahunas, that battle for her favour.',
  0.30319542]]

In [ ]:
result=topNSimilar(claim, res[0])

In [ ]:
titles= [i.title for i in res[1]]

In [ ]:
titles

['Death of Subhas Chandra Bose',
 'Subhas Chandra Bose',
 'Aviation accidents and incidents',
 'Ritchie Valens',
 'Dior',
 'Bachelor of Arts',
 'Lynyrd Skynyrd plane crash']

In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
data = res[0]

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
max_epochs = 80
vec_size = 30
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=2,
                dm =1,
                epochs= max_epochs)
  
model.build_vocab(tagged_data)



NameError: ignored

In [ ]:
model.train(tagged_data, total_examples=model.corpus_count, epochs=max_epochs)

test_data = word_tokenize(claim.lower())
v1 = model.infer_vector(test_data)

# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('1')
# print(similar_doc)


# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
# print(model.docvecs['1'])


In [ ]:
for i in range(5):
    print(data[int(similar_doc[i][0])])

Chattopadhyay was very intereste in recent events in Indian and Bengali history, particularly the Revolt of 1857 and the previous century's Sanyasi Rebellion.
The poem was published in Chattopadhyay's book Anandamath (pronounced Anondomôţh in Bengali) in 1882, which is set in the events of the Sannyasi Rebellion.
from Bengali by Surendranath Tagore.
Durgeshnandini, his first Bengali romance and the first ever novel in Bengali, was published in 1865.
His first attempt was a novel in Bengali submitted for a declared prize.


In [41]:
def topNBert(res):
    topNFacts=[i[0] for i in res]
    topNScore=[cosine_similarity( [model.encode(claim)], [model.encode(i)] )[0][0] for i in topNFacts]
    topN=zip(topNFacts, topNScore)
    return [list(i) for i in list(topN)]